In [231]:
import numpy as np
import datasets

datasets.downloads = '/home/greg/Downloads/'

In [310]:
trX, teX, trY, teY = datasets.mnist()

In [312]:
trX, trY = datasets.htwt()
trY = one_hot(trY)

In [233]:
decode = lambda p: np.argmax(p, axis = 1) 
accuracy = lambda p, Y: np.mean(decode(p) == decode(Y))

def one_hot(y):
    y = np.array(y)
    if y.ndim == 2:
        return y
    else:
        N, C = len(y), len(np.unique(y))
        Y = np.zeros((N, C))
        for i in range(N):
            Y[i, int(y[i])] = 1
        return Y

In [294]:
import tensorflow as tf

def dense(output_dim, x):
    shape = (x.get_shape()[1].value, output_dim)
    w = tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    b = tf.Variable(tf.zeros(shape[1], dtype = tf.float32))
    return tf.matmul(x, w) + b  
    
def fit(sess, x, y, X, Y, optimizer, epochs):
    sess.run(tf.initialize_all_variables())
    N, D = X.shape

    for _ in range(epochs):
        for start in range(0, N - 32, 32):
            end = start + 32
            feed_dict = {x: X[start : end], y: Y[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)
            
x = tf.placeholder(tf.float32, (None, trX.shape[1]))
y = tf.placeholder(tf.float32)
h = tf.nn.relu(dense(625, x))  
model = dense(trY.shape[1], h)
loss = tf.nn.softmax_cross_entropy_with_logits(model, y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    fit(sess, x, y, trX, trY, optimizer, epochs = 1)
    print accuracy(model.eval({x: teX}), teY)

0.9629


In [311]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(output_dim = 625, input_dim = trX.shape[1], activation = 'relu'))
model.add(Dense(output_dim = trY.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(trX, trY, nb_epoch = 1)
print accuracy(model.predict_proba(teX, verbose = 0), teY)

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.1948    
0.9681


In [7]:
config = model.get_config()
weights = model.get_weights()

model = Sequential.from_config(config)
model.set_weights(weights)

In [ ]:
# https://github.com/Newmu/Theano-Tutorials

import theano
from theano import tensor as T

x = T.fmatrix()
y = T.fmatrix()

weights = lambda n, d: theano.shared(np.random.randn(n, d))

w_h = weights(D, H)
w_o = weights(H, C)

l1 = T.nnet.sigmoid(T.dot(x, w_h))
l2 = T.nnet.softmax(T.dot(l1, w_o))

cost = T.mean(T.nnet.categorical_crossentropy(l2, y))
updates = [[p, p - T.grad(cost, p) * learning_rate] for p in [w_h, w_o]]
optimizer = theano.function([x, y], updates = updates, allow_input_downcast = True)

for _ in range(epochs):
    for start, end in zip(range(0, N, 128), range(128, N, 128)):
        optimizer(X[start : end], Y[start : end])
        
w_h, w_o = w_h.get_value(), w_o.get_value()
print accuracy(softmax(expit(X.dot(w_h)).dot(w_o)))

# you can play with theano to write derivative functions

x = T.fmatrix()
y = T.fmatrix()
loss = T.mean(T.nnet.categorical_crossentropy(T.nnet.softmax(x), y))
grad_loss = theano.function([x, y], T.grad(loss, x))

x = [[-3, 3], [-2, 1], [-2, 1]]
y = [[0.0, 1], [0, 1], [1, 0]]
grad_loss(x, y)

In [463]:
import numpy as np
from softmax import softmax

epochs = 1
batch_size = 32
learning_rate = 0.001
# maybe preprocess X
R = lambda Ws: 0 # regularization penalty
H = 100
init_weights = lambda fan_in, fan_out: np.random.randn(fan_in, fan_out) * 0.05 # try glorot

N, D = trX.shape
N, C = trY.shape

W1 = init_weights(D, H)
b1 = np.zeros(H)
W2 = init_weights(H, C)
b2 = np.zeros(C)

def fit(b2):
    global W1, b1, W2
    
    for _ in xrange(epochs):
        for start in xrange(0, len(trX) - batch_size + 1, batch_size):
            end = start + batch_size
            X = trX[start : end]
            Y = trY[start : end]
            n = len(X)

            # forward propagate
            N1 = X.dot(W1) + b1
            H1 = np.maximum(0, L1)

            N2 = H1.dot(W2) + b2
            H2 = N2

            sm = softmax(H2)
            loss = -np.mean(np.log(np.sum(sm * Y, axis = 1)))

            # back propagate and update
            grad_loss = (sm - Y) / n 

            grad_H2 = grad_loss
            grad_W2 = H1.T.dot(grad_H2)
            grad_b2 = np.sum(grad_H2, axis = 0)

            grad_H1 = grad_scores.dot(W2.T)
            grad_N1 = N1
            grad_N1[grad_N1 >= 0] = 1
            grad_N1[grad_N1 < 0] = 0
            grad_W1 = np.dot(X.T, grad_H1 * grad_N1)
            grad_b1 = np.sum(grad_H1 * grad_N1, axis = 0)
            break

            W2 -= grad_W2 * learning_rate
            b2 -= grad_b2 * learning_rate
            W1 -= grad_W1 * learning_rate
            b1 -= grad_b1 * learning_rate
    print grad_b2[:5]
    return loss

print numerical_gradient(fit, b2)[:5]

[-0.68629131  0.68629131]
[-0.68629133  0.68629133]
[-0.68629133  0.68629133]
[-0.68629131  0.68629131]
[-0.68629132  0.68629132]


In [437]:
def numerical_gradient(f, x):
    h = 0.00001
    grad = np.zeros_like(x)
    it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    
    while not it.finished:
        ix = it.multi_index
        oldval = x[ix]
        x[ix] = oldval + h
        fxph = f(x)
        x[ix] = oldval - h
        fxmh = f(x)
        x[ix] = oldval
        grad[ix] = (fxph - fxmh) / (2 * h)
        it.iternext()
        
    return grad

x = np.array([1, 2])
f = lambda x: np.sum(x)
numerical_gradient(f, x)

array([49999, 49999])